In [16]:
# pip install geopandas rasterio shapely pyproj numpy pandas
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape as shp_shape
import rasterio
from rasterio.transform import from_origin
from rasterio.features import shapes
from rasterio.crs import CRS

# --- user paths ---
csv_path = r"./spatial_interpolation.csv"
tif_out  = r"./rain_idw.tif"
shp_out  = r"./rain_polygons.shp"   # fixed path

# --- columns ---
x_field = "Longitude"
y_field = "Latitude"
z_field = "Rainfall(mm)"

# --- CRS ---
crs_wgs84 = "EPSG:4326"
crs_proj  = "EPSG:32645"  # UTM 45N for Nepal area

# 1) Read CSV -> GeoDataFrame in WGS84
df = pd.read_csv(csv_path)
gdf = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df[x_field], df[y_field])],
    crs=crs_wgs84
)

# 2) Reproject to meters (UTM)
gdf_m = gdf.to_crs(crs_proj)

# 3) Build a grid over the extent
ext = gdf_m.total_bounds  # [minx, miny, maxx, maxy]
padx = (ext[2] - ext[0]) * 0.10
pady = (ext[3] - ext[1]) * 0.10
minx, miny, maxx, maxy = ext[0] - padx, ext[1] - pady, ext[2] + padx, ext[3] + pady

# grid resolution in meters (adjust as needed)
res = 250.0
nx = int(np.ceil((maxx - minx) / res))
ny = int(np.ceil((maxy - miny) / res))

# coordinates of grid cell centers
xs = minx + (np.arange(nx) + 0.5) * res
ys = maxy - (np.arange(ny) + 0.5) * res  # top-down

# 4) Prepare point arrays for IDW
px = gdf_m.geometry.x.values
py = gdf_m.geometry.y.values
pz = gdf_m[z_field].values

# 5) Simple IDW implementation
power = 2.0
eps   = 1e-12

Z = np.zeros((ny, nx), dtype="float32")
for j in range(ny):
    y = ys[j]
    dx = xs[None, :] - px[:, None]  # (npoints, nx)
    dy = y - py[:, None]            # (npoints, nx)
    d2 = dx*dx + dy*dy
    w  = 1.0 / np.maximum(d2, eps)**(power/2.0)

    # weighted mean
    Zj = np.sum(w * pz[:, None], axis=0) / np.sum(w, axis=0)

    # handle exact hits (distance ~ 0): set to station value
    exact_cols = np.where((d2 < 1e-8).any(axis=0))[0]
    for c in exact_cols:
        r = np.where(d2[:, c] < 1e-8)[0][0]
        Zj[c] = pz[r]

    Z[j, :] = Zj.astype("float32")

# 6) Write GeoTIFF (continuous surface)
transform = from_origin(minx, maxy, res, res)
with rasterio.open(
    tif_out,
    "w",
    driver="GTiff",
    height=Z.shape[0],
    width=Z.shape[1],
    count=1,
    dtype=Z.dtype,
    crs=CRS.from_string(crs_proj),
    transform=transform,
) as dst:
    dst.write(Z, 1)

# 7) Build appropriate rainfall classes (25 mm bins snapped to data)
step = 25.0  # change to 10/20/50 if you prefer
zmin = float(np.nanmin(Z))
zmax = float(np.nanmax(Z))
start = np.floor(zmin / step) * step
end   = np.ceil(zmax / step) * step
bins  = np.arange(start, end + step, step, dtype="float32")

# classify and write a class raster (use uint16 in case classes >255)
Zb = np.digitize(Z, bins, right=False).astype("uint16")
with rasterio.open(
    tif_out.replace(".tif", "_classes.tif"),
    "w",
    driver="GTiff",
    height=Zb.shape[0],
    width=Zb.shape[1],
    count=1,
    dtype=Zb.dtype,
    crs=CRS.from_string(crs_proj),
    transform=transform,
) as dst:
    dst.write(Zb, 1)

# polygonize classes to vector
mask = np.ones_like(Zb, dtype=np.uint8)
polys, vals = [], []
for geom, val in shapes(Zb, mask=mask, transform=transform):
    if val == 0:  # below first bin
        continue
    polys.append(shp_shape(geom))
    vals.append(int(val))

vals_arr  = np.array(vals, dtype="int32")
class_min = bins[np.clip(vals_arr - 1, 0, len(bins) - 1)]
class_max = bins[np.clip(vals_arr,     0, len(bins) - 1)]

poly_gdf = gpd.GeoDataFrame(
    {
        "class_id": vals_arr,
        "class_min": class_min.astype(float),
        "class_max": class_max.astype(float),
    },
    geometry=polys,
    crs=crs_proj
)

# Friendly label for QGIS legends
poly_gdf["label"] = (
    poly_gdf["class_min"].round(0).astype(int).astype(str)
    + "–" +
    poly_gdf["class_max"].round(0).astype(int).astype(str)
    + " mm"
)

# Save shapefile
poly_gdf.to_file(shp_out)

print("Done. Outputs:")
print("Continuous raster:", tif_out)
print("Class raster:", tif_out.replace(".tif", "_classes.tif"))
print("Class polygons:", shp_out)


Done. Outputs:
Continuous raster: ./rain_idw.tif
Class raster: ./rain_idw_classes.tif
Class polygons: ./rain_polygons.shp
